# Step-1



# Importing Requried Library

In [ ]:
## Importing Library

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
%matplotlib inline
# Plot Style
sns.set_context("paper")
style.use('fivethirtyeight')

## Sklearn library
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import precision_score, recall_score,classification_report,accuracy_score,confusion_matrix,roc_auc_score,roc_curve,auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score

## Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Step-2

# Setting up Jupyter View

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# Step-3

# Importing the Input file

In [ ]:
lead = pd.read_csv("Lead Scoring.csv")
lead.head()

In [ ]:
##Inspect Dataframe

#database dimension
print("Database dimension     :",lead.shape)
print("Database size          :",lead.size)
print("Number of Row          :",len(lead.index))
print("Number of Columns      :",len(lead.columns))

In [ ]:
#checking numerical columns statistics
lead.describe()

In [ ]:
#info about the column types etc. 
lead.info()

# Null Value Calculation

In [ ]:
#Column wise Null Value calculate
#Column wise null values in train data set 
null_perc = pd.DataFrame(round((lead.isnull().sum())*100/lead.shape[0],2)).reset_index()
null_perc.columns = ['Column Name', 'Null Values Percentage']
null_value = pd.DataFrame(lead.isnull().sum()).reset_index()
null_value.columns = ['Column Name', 'Null Values']
null_lead = pd.merge(null_value, null_perc, on='Column Name')
null_lead.sort_values("Null Values", ascending = False)

In [ ]:
#Column wise Null Value calculate
#plotting the null value percentage
sns.set_style("white")
fig = plt.figure(figsize=(12,5))
null_lead = pd.DataFrame((lead.isnull().sum())*100/lead.shape[0]).reset_index()
ax = sns.pointplot("index",0,data=null_lead)
plt.xticks(rotation =90,fontsize =9)
ax.axhline(45, ls='--',color='red')
plt.title("Percentage of Missing values")
plt.ylabel("PERCENTAGE")
plt.xlabel("COLUMNS")
plt.show()

# Duplicate Analysis

In [ ]:
## Hint--- check for ID,columns
print("Total number of duplicate values in Prospect ID column :" , lead.duplicated(subset = 'Prospect ID').sum())
print("Total number of duplicate values in Lead Number column :" , lead.duplicated(subset = 'Lead Number').sum())
print("Total number of duplicate values in Lead Origin column :" , lead.duplicated(subset = 'Lead Origin').sum())

# Data Cleaning

In [ ]:
## drop those column which contain more than 50% nulll values.

## Drop those column which has zero duplicate values/ uniquness.

## Drop the useless columns which are not requried in our Model Development

## How to drop
## Ex-
cols_to_drop = ['Prospect ID','Lead Number','How did you hear about X Education','Lead Profile',
                'Lead Quality','Asymmetrique Profile Score','Asymmetrique Activity Score',
               'Asymmetrique Activity Index','Asymmetrique Profile Index','Tags','Last Notable Activity']

## Dropping the unnecessary columns
lead.drop(cols_to_drop,axis=1,inplace=True)
lead.head()

len(lead.columns)


_______________________________________________________________________________________________________________________________

## Tips
## Replace values and Imputing codes
lead['Lead Source'] = lead['Lead Source'].replace('google', 'Google')
lead['Lead Source']=lead['Lead Source'].replace(np.nan,"Email Opened")
lead['TotalVisits']=lead['TotalVisits'].fillna(lead['TotalVisits'].median())

# Renaming some of the column headers which has long header
lead.rename(columns={"What is your current occupation":'Occupation',"A free copy of Mastering The Interview":"Free Copy",
                     "Through Recommendations":"Recommendation"},inplace=True)
## Groupby
top_10_happiest=df[:10].groupby('Country')['Happiness Rank'].max().sort_values(ascending=True)


#Binning
## MAking a group of tenure based of a Year
Labels=["{0}-{1}".format(i,i+11) for i in range(1,72,12)]

df['tenure_group']=pd.cut(df['tenure'],range(1,80,12),right=False,labels=Labels)

## Set a cutoff to convert multiclass class classification to Binary Class.
bins=[2,6,9]
labels=['Bad','Good']
df_1['quality']=pd.cut(df_1['quality'],bins=bins,labels=labels)
df_1.head()

# Segregating Categorical & Numerical Values

In [ ]:
categorical_col=lead.select_dtypes(exclude=['number']).columns.values
numerical_col=lead.select_dtypes(include=['number']).columns.values


print("Categorical Features:\n{}".format(categorical_col))
print("Numerical Features:\n{}".format(numerical_col))

# Categorical Column Analysis

In [ ]:
############ Below Function will give You values,unique_values,null vaules, null_perent in  a dataframe###########
def Cat_info(df,categorical_column):
    df_result=pd.DataFrame(columns=["columns","values","unique_values","null_values","null_percent"])
    
    df_temp=pd.DataFrame()
    for value in categorical_column:
        df_temp['columns']=[value]
        df_temp['values']=[df[value].unique()]
        df_temp["unique_values"]=df[value].nunique()
        df_temp["null_values"] = df[value].isna().sum()
        df_temp["null_percent"] = (df[value].isna().sum()/len(df)*100).round(1)
        df_result = df_result.append(df_temp)
    df_result.sort_values("null_values", ascending =False, inplace=True)
    df_result.set_index("columns", inplace=True)
    return df_result
####################
df_cat = Cat_info(lead, categorical_col)
df_cat

## Hint:: We can check here catrgorical column with 1 unique values or multiple values and furter we can drop in our analysis

# Appending the columns to col_to_drop where only 1 category value is present

cols_to_drop = df_cat[df_cat['unique_values']==1].index.values.tolist() 
cols_to_drop

#dropping unnecessary columns(Like unique values ==1)

lead.drop(cols_to_drop, 1, inplace = True)
len(lead.columns)  ## Here we dropped columns from our primary dataset

## Again creating the categorical col and checking indepth analysis for Null values and Categorical Columns
categorical_col = lead.select_dtypes(exclude =["number"]).columns.values
new_cat = Cat_info(lead, categorical_col)
new_cat


## Add on

## Chekcing Value counts of all the object attributes.
for col in obj_df:
    print('--------------',col,'------------------------')
    print(obj_df[col].value_counts())

# Null Values Handling in Categorical Columns

In [ ]:
(# Do this value counts for all the categorical Features)
lead['City'].value_counts(normalize=True)*100 
## Plotting the Value Counts
style.use('fivethirtyeight')
sns.countplot(lead['City'])
plt.xticks(rotation=90)
plt.show()
## Tips:-

#1- In above null value percentage we can compare and we can't put Mode in every categorical null values.
#2- If in any categorical column null values in more then 40 % then if we put Mode values, If will create a skewness.
#3- In this case Either we can create a New Category and put that in place of null values 
lead['Specialization']=lead['Specialization'].replace(np.nan,"Others")
plt.figure(figsize=(12,6))
sns.countplot(lead['Specialization'],palette = 'Set2')
plt.xticks(rotation = 90)
plt.show()
###
#4- If we visualize or in value counts, then are values are more (90%) belong to one category, then we can consider
#   it as a skewed. And We can drop it.
#5-  While impute mode, we should always care that our feature should not be skewed. So choose wisely. Put any random 'category'.

print("Number of null values present in Last Activity column is",lead['Last Activity'].isnull().sum())
print("Percentage of Null values present in Last Activity Columns is",(lead['Last Activity'].isnull().sum())*100/lead.shape[0])    

In [ ]:
## Imputing most occuring value or Mode value

lead['Last Activity']=lead['Last Activity'].replace(np.nan,"Email Opened")
#or
df['last Activity'] = df['Last Activity'].fillna(df['Last Activity'].mode()[0])

In [ ]:
## Again Checking null Values
categorical_col = lead.select_dtypes(exclude =["number"]).columns.values
new_cat_null = Cat_info(lead, categorical_col)
new_cat_null

## This is final Stage for Categorical Feature columns
#1- Make sure there is no missing values anymore
#2- Check the unique values, replace if any spelling mistakes are present.


# Numerical Columns Null Value Treatment

In [ ]:
# Checking unique values and null values for the numerical columns
def Num_info(df, numeric_column):
    df_result = pd.DataFrame(columns=["columns","null_values","null_percent","Skewness"])
    
    df_temp=pd.DataFrame()
    for value in numeric_column:
        df_temp["columns"] = [value]
        df_temp["Skewness"] = df[value].skew()
        df_temp["null_values"] = df[value].isna().sum()
        df_temp["null_percent"] = (df[value].isna().sum()/len(df)*100).round(1)
        df_result = df_result.append(df_temp)
    
    df_result.sort_values("null_values", ascending =False, inplace=True)
    df_result.set_index("columns", inplace=True)
    return df_result

df_num = Num_info(lead,numerical_col)
df_num


In [ ]:
## Plot this graph for all the features . This piece of code will give you KDE and Boxplot. Do it for all the columns
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
sns.distplot(lead['TotalVisits'])
plt.subplot(1,2,2)
sns.boxplot(lead['TotalVisits'])

## Tips
#1- If by seeing the in above data, we can deside. If outlier are present then impute median value in place of missing values.
lead['TotalVisits']=lead['TotalVisits'].fillna(lead['TotalVisits'].median())

# 5. Exploratory Data Analysis

In [ ]:
## For target column( category and Imbalance Check)
labels = 'No','Yes'
lead['Converted'].astype(str).value_counts().plot(kind='pie',
                            figsize=(10, 6),
                            autopct='%1.1f%%', 
                            startangle=90,    
                            shadow=True,       
                            labels=None,                                
                            )

plt.title('Distribution of Lead Approval', y=1.12) 
plt.axis('equal') 
# add legend
plt.legend(labels=labels, loc='upper left') 
 # show plot
plt.show()

# Univariate Analysis - Categorical

In [ ]:
converted_lead=lead[lead['Converted']==1]
fail_lead=lead[lead['Converted']==0]

In [ ]:
## This Function will Give a good comparision if we have binary class classifiaction(churn/Not)
def univariate_categorical(column):
    check=pd.DataFrame()
    check['Total Leads']=lead[column].value_counts()
    check['Qualified Lead']=converted_lead[column].value_counts()
    check['Non Qualified Lead']=fail_lead[column].value_counts()
    check["% Lead Converted"]=round(check['Qualified Lead']*100/check['Total Leads'],2)
    check=check.fillna(0)
    #check.columns=['Leads','Total Leads','Qualified Lead',"Non Qualify Lead","% Lead Converted"]
    check.reset_index(inplace=True)## 
    check=check.rename(columns={"index":column})
    
    
    plt.figure(figsize=(15,8))
    plt.subplot(1,2,1)
    plt.title(column)
    plt.xticks(rotation = 90)
    sns.countplot(lead[column],hue=lead['Converted'])

    plt.subplot(1,2,2)
    plt.title("% of Lead Conversion")
    plt.xticks(rotation = 90)
    sns.barplot(x=check[column],y=check['% Lead Converted'],data=lead)
                                           
    return check

# Numerical Column for Outliers Treatment

# Multiple Box plot and Distributon plot for Outlier Detection

In [ ]:
### How to plot multiple distribution plot##
plt.figure(figsize=(18,10))
plot=1
for col in num_df:
    if plot<=6:
        plt.subplot(2,3,plot)
        sns.distplot(df[col],color='red')
        plt.xlabel(col)
        plot=plot+1
plt.show()

## For boxplot

plt.figure(figsize=(18,10))
plot=1
for col in num_df:
    if plot<=6:
        plt.subplot(2,3,plot)
        sns.boxplot(df[col],color='green')
        plt.xlabel(col)
        plot=plot+1
plt.show()

# Outliers Treatment

In [ ]:
## when data is normally distributed.(IQR Methood). Best Approach
def replace_outlier(df,col):
    IQR=df[col].quantile(.75)-df[col].quantile(.25)
    lower_limit=df[col].quantile(.25)-(1.5*IQR)
    upper_limit=df[col].quantile(.75)+(1.5*IQR)
    non_outlier=np.where((df[col]<lower_limit )|(df[col]>upper_limit),df[col].median(),df[col])
    df[col]=non_outlier
    plt.subplot(1,2,1)
    sns.distplot(df[col])
    plt.subplot(1,2,2)
    sns.boxplot(df[col])
    
## If our data is Skewed (Quantile Method)
def outlier_IQR(data,col):
    IQR=data[col].quantile(.75)-data[col].quantile(.25)
    lower_bridge=data[col].quantile(.25)-(IQR*1.5)
    upper_bridge=data[col].quantile(.75)+(IQR*1.5)
    return (lower_bridge,upper_bridge)

# if our data is normally distributed 
def outlier_normally(df,col):
     Lower_boundary=df[col].mean()*-3*df[col].std()
     upper_boundary=df[col].mean()* +3*df[col].std()
     return (Lower_boundary,upper_boundary) 
    
    
#Example of outliers Treatment
def replace_outlier(df,col):
    IQR=df[col].quantile(.75)-df[col].quantile(.25)
    lower_limit=df[col].quantile(.25)-(1.5*IQR)
    upper_limit=df[col].quantile(.75)+(1.5*IQR)
    non_outlier=np.where((df[col]<lower_limit )|(df[col]>upper_limit),df[col].median(),df[col])
    df[col]=non_outlier
    
    
#replotting the graphs to check for outlier treatment
i=1
plt.figure(figsize=[16,8])
for col in numerical_col:
    plt.subplot(2,2,i)
    sns.boxplot(y=lead[col])
    plt.title(col)
    plt.ylabel('')
    i+=1

In [ ]:
## Again  Checking the null values 
numerical_col = lead.select_dtypes(include=["number"]).columns.values
new_num_null = Num_info(lead, numerical_col)
new_num_null

# Skewness Check and removal

In [ ]:
## removing skewness
from scipy import stats
df['Diameter'],parameters=stats.boxcox(df['Diameter'])
df['Length'],parameters=stats.boxcox(df['Length'])

##Log transformation
def log_transform(df,col):
    df_1=df.copy()
    if 0 in df_1[col].unique():
        pass
    else:
        df_1[col]=np.log(df_1[col])
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        stats.probplot(df_1[col],dist='norm',plot=pylab)
        plt.subplot(1,2,2)
        df[col].hist()
        plt.title('log_transfer')
##Reciprocal_transform
def reciprocal_transform(df,col):
    df_1=df.copy()
    if 0 in df_1[col].unique():
        pass
    else:
        df_1[col]=1/df_1[col]
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        stats.probplot(df_1[col],dist='norm',plot=pylab)
        plt.subplot(1,2,2)
        df[col].hist()
        plt.title('log_transfer')
        
## Log 1p transfromation
def log1p_transform(df,col):
    df_1=df.copy()
    if 0 in df_1[col].unique():
        pass
    else:
        df_1[col]=np.log1p(df_1[col])
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        stats.probplot(df_1[col],dist='norm',plot=pylab)
        plt.subplot(1,2,2)
        df[col].hist()
        plt.title('log_transfer')
    
##Exponential transformation Technique
def exponential_transform(df,col):
    df_1=df.copy()
    if 0 in df_1[col].unique():
        pass
    else:
        df_1[col]=df_1[col]**(1/1.2)
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        stats.probplot(df_1[col],dist='norm',plot=pylab)
        plt.subplot(1,2,2)
        df[col].hist()
        plt.title('log_transfer')
        
## Squareroot transformation
def squareroot_transform(df,col):
    df_1=df.copy()
    if 0 in df_1[col].unique():
        pass
    else:
        df_1[col]=np.sqrt(df_1[col])
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        stats.probplot(df_1[col],dist='norm',plot=pylab)
        plt.subplot(1,2,2)
        df[col].hist()
        plt.title('log_transfer')
## cuberoot Transformation
def cuberoot_transform(df,col):
    df_1=df.copy()
    if 0 in df_1[col].unique():
        pass
    else:
        df_1[col]=np.cbrt(df_1[col])
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        stats.probplot(df_1[col],dist='norm',plot=pylab)
        plt.subplot(1,2,2)
        df[col].hist()
        plt.title('log_transfer')
        
##Boxcox transformation
from scipy import stats
def boxcox_transform(df,col):
    df_1=df.copy()
    if 0 in df_1[col].unique():
        pass
    else:
        df_1[col],parameter=stats.boxcox(df_1[col])
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        stats.probplot(df_1[col],dist='norm',plot=pylab)
        plt.subplot(1,2,2)
        df[col].hist()
        plt.title('log_transfer')

In [ ]:
lead=lead.replace({"Yes": 1, "No":0})

# Encoding

In [ ]:
## One Hot Encoding
lead=pd.get_dummies(lead,columns=['Lead Origin','Lead Source','Last Activity','Specialization','Occupation'],drop_first=True)

## Label Encoding
from sklearn.preprocessing import LabelEncoder
 le=LabelEncoder()
for col in lead.columns:
    if lead[col].dtype=='object':
         lead[col]=le.fit_transform(lead[col])
            
## target Encoding
import category_encoders as ce
encoder=ce.TargetEncoder()
for col in lead.columns:
    if lead[col].dtype=='object':
        lead[col]=encoder.fit_transform(lead[col],lead['Converted'])

# HeatMap

In [ ]:
# Visualizing the data using heatmap to check Correlation
plt.figure(figsize=[15,15])
sns.heatmap(lead.corr(), cmap="RdYlGn",linewidth =1)
plt.show()


## Correlation bar plot
lead.corr()['Converted'].sort_values(ascending=False).drop(['Converted']).plot.bar(figsize=(10,6),grid=True,title='Correlation With target',fontsize=15)


#or

## Correlation with Happiness score
df.corr()['Happiness Score']

In [ ]:
## Most Postitive Correlated 
corr_lead = lead.corr()
corr_lead = corr_lead.where(np.triu(np.ones(corr_lead.shape),k=1).astype(np.bool))
corr_df = corr_lead.unstack().reset_index()
corr_df.columns =['VAR1','VAR2','Correlation']
corr_df.dropna(subset = ["Correlation"], inplace = True) 
corr_df.sort_values(by='Correlation', ascending=False, inplace=True)

# Top 5 Positive correlated variables
corr_df.head(5)


corr_df.sort_values(by='Correlation', ascending=True, inplace=True)

# Top 5 Negatively correlated variables
corr_df.head(5)

# Train - Test Split

In [ ]:
# target variable
Y = lead['Converted']
X = lead.drop(['Converted'], axis=1)


# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.7, test_size=0.3, random_state=100)

In [ ]:
#Checking the shape of the created Train & Test DFs
print(" Shape of X_train is : ",X_train.shape)
print(" Shape of y_train is : ",y_train.shape)
print(" Shape of X_test is  : ",X_test.shape)
print(" Shape of y_test is  : ",y_test.shape)

In [ ]:
## Alternative

# Creating a test set X_test, then X_valid becomes our validation set
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.8, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_rem, y_rem, test_size=0.5)


rf = RandomForestClassifier(n_estimators=100, criterion="entropy", random_state=0)
rf = rf.fit(X_train, y_train)


print('****Results****')
train_predictions = rf.predict(X_valid)
acc = accuracy_score(y_valid, train_predictions)
print(classification_report(train_predictions, y_valid))
print("Accuracy: {:.4%}".format(acc))

## Predicting on Unseen Data
print('****Results****')
train_predictions = rf.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print(classification_report(train_predictions, y_test))
print("Accuracy: {:.4%}".format(acc))

# Feature Scaling

In [ ]:
# data normalization with sklearn
from sklearn.preprocessing import StandardScaler

# fit scaler on training data
ss = StandardScaler().fit(X_train)

# transform training data
X_train_norm = ss.transform(X_train)

# transform testing dataabs
X_test_norm = ss.transform(X_test)

# Feature Selection

In [ ]:
##This Method we can we can use in case of Multicollinearity 

## Referecne : https://www.youtube.com/watch?v=FndwYNcVe0U&list=PLZoTAELRMXVPgjwJ8VyRoqmfNs2CJwhVH&index=2

import seaborn as sns
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = X_train.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.CMRmap_r)
plt.show()




# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):   ## In dataset only Put X_train
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

## to get number of highly correlated columns
corr_features = correlation(X_train, 0.7)
len(set(corr_features))

## Getting the names of the particular correlated features
corr_features


## Dropping from X_train and Y_train. After dropping , mmodel is ready to build.
X_train.drop(corr_features,axis=1)
X_test.drop(corr_features,axis=1)

In [ ]:
#3-Feature Selection-Information gain - mutual information In Classification Problem Statements
## Importing the library
from sklearn.feature_selection import mutual_info_classif
# determine the mutual information
mutual_info = mutual_info_classif(X_train, y_train)
mutual_info

##Getting values
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

###let's plot the ordered mutual_info values per feature
mutual_info.sort_values(ascending=False).plot.bar(figsize=(20, 8))

## Selecting the k best parameters
from sklearn.feature_selection import SelectKBest

#No we Will select the  top 5 important features
sel_five_cols = SelectKBest(mutual_info_classif, k=5)
sel_five_cols.fit(X_train, y_train)
X_train.columns[sel_five_cols.get_support()]


## The output columns of this will help me to select the columns for our model building

In [ ]:
#Feature Selection-Information gain - mutual information In Regression Problem Statements

##Code-1
from sklearn.feature_selection import mutual_info_regression
# determine the mutual information
mutual_info = mutual_info_regression(X_train, y_train)
mutual_info

## Code-2
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

##Code-3

mutual_info.sort_values(ascending=False).plot.bar(figsize=(15,5))

#Code-4
#No we Will select the  top 5 important features
sel_five_cols = SelectKBest(mutual_info_classif, k=5)
sel_five_cols.fit(X_train, y_train)
X_train.columns[sel_five_cols.get_support()]

# Feature Importance

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model=ExtraTreesClassifier()
model.fit(X,y)


print(model.feature_importances_)


ranked_features=pd.Series(model.feature_importances_,index=X.columns)

ranked_features.nlargest(10).plot(kind='barh')

# PCA

In [ ]:
from sklearn import decomposition
from sklearn.decomposition import PCA
covar_matrix=PCA(n_components=34)


#Calculate Eigenvalues
covar_matrix.fit(x)  ## x should be scaled
variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios

var=np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3)*100)
var #cumulative sum of variance explained with [n] features
## draw the graph
plt.ylabel('% Variance Explained')
plt.xlabel('# of Features')
plt.title('PCA Analysis')
plt.ylim(34,100.5)
plt.style.context('seaborn-whitegrid')


plt.plot(var)

In [ ]:
pca=PCA(n_components="Based on above graph stagnation point")
x=pca.fit_transform(x)
x.shape

# Model Building

In [ ]:
## For Classification
score=[]
f_1=[]
CV_score=[]
model_name=[]
SD=[]
def best_model(clf):
    clf.fit(X_train_norm,y_train)
    print(clf)
    print('Trainng Score:',clf.score(X_train_norm,y_train))
    print('Testing score:',clf.score(X_test_norm,y_test))
    pred=clf.predict(X_test_norm)
    print('Accuracy_score',accuracy_score(y_test,pred))
    print('F1 score',f1_score(y_test,pred))
    print('cross validation score',cross_val_score(clf,X,Y,scoring='accuracy').mean())
    print('Standard Deviation',cross_val_score(clf,X,Y,scoring='accuracy').std())
    print('confusion matrix',confusion_matrix(y_test,pred))
    print('\n')
    print('Classification Report',classification_report(y_test,pred))
    print('---'*20)
    print('Roc_auc Score',roc_auc_score(y_test,pred))
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,pred)
    roc_auc = auc( false_positive_rate, true_positive_rate)
    plt.plot(false_positive_rate, true_positive_rate,label = "AUC = %0.2f"% roc_auc)
    plt.plot([0,1],[0,1],'r--')
    plt.legend(loc = 'lower right')
    plt.ylabel("True positive rate")
    plt.xlabel("False positive rate")
    print("\n\n")
    score.append(accuracy_score(y_test,pred))
    CV_score.append(cross_val_score(clf,X,Y,scoring='accuracy').mean())
    f_1.append(f1_score(y_test,pred))
    SD.append(cross_val_score(clf,X,Y,scoring='accuracy').std())

In [ ]:
## For imbalance dataset

## Over sampling our data
sm = SMOTEENN()
X_resampled1, y_resampled1 = sm.fit_resample(x,y)  ## Where X is Independent features and Y is dependent Features

def max_accuracy_score(clf,x,y):
    max_accuracy=0
    for i in range(42,100):
        x_train,x_test,y_train,y_test=train_test_split(X_resampled1,y_resampled1,test_size=.20,random_state=i,stratify=y_resampled1)
    ##over sampling of our Data
       # print(x_train.shape,y_train.shape) 
        clf.fit(x_train,y_train)
        pred=clf.predict(x_test)
        accuracy_check=accuracy_score(y_test,pred)
        if accuracy_check>max_accuracy:
            max_accuracy=accuracy_check
            final_r=i
    print('max accuracy score corresponding to',final_r,'is',max_accuracy)
    print('\n')
    print('cross validation score',cross_val_score(clf,x,y,scoring='accuracy').mean())
    print('\n')
    print('Standard Deviation',cross_val_score(clf,x,y,scoring='accuracy').std())
    print('\n')
    print('F1 score',f1_score(y_test,pred))
    print('\n')
    print('Training accuracy',clf.score(x_train,y_train))
    print('\n')
    print('Test Accuracy',clf.score(x_test,y_test))
    print('\n')
    print('Confusion Matrix',confusion_matrix(y_test,pred))
    print('\n')
    print('Classification Report',classification_report(y_test,pred))
    print('\n')
    print('Roc_auc Score',roc_auc_score(y_test,pred))
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,pred)
    roc_auc = auc( false_positive_rate, true_positive_rate)
    plt.plot(false_positive_rate, true_positive_rate,label = "AUC = %0.2f"% roc_auc)
    plt.plot([0,1],[0,1],'r--')
    plt.legend(loc = 'lower right')
    plt.ylabel("True positive rate")
    plt.xlabel("False positive rate")
    print("\n\n")
    return final_r

In [ ]:
### For regression
def maxr2_score(clf,x,y):
    maxr2_score1=0
    for i in range(42,100):
        x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=i)
      ## incase of imbalanced dataset x_train_s,y_train_s=SMOTE().fit_resample(x_train,y_train)
        clf.fit(x_train,y_train)
        pred=clf.predict(x_test)
        r2_cscore=r2_score(y_test,pred)
        if r2_cscore>maxr2_score1:
            maxr2_score1=r2_cscore
            final_r=i
    print('max r2 score corresponding to',final_r,'is',maxr2_score1)
    print('cross validation score',cross_val_score(clf,x,y,scoring='r2').mean())
    print('Standard Deviation',cross_val_score(clf,x,y,scoring='r2').std())
    print('Training accuracy',clf.score(x_train,y_train))
    print('Test Accuracy',clf.score(x_test,y_test))
    print('MAE',mean_absolute_error(y_test,pred))
    print('MSE',mean_squared_error(y_test,pred))
    print('RMSE',np.sqrt(mean_squared_error(y_test,pred)))
    
    return final_r

In [ ]:
lr=LogisticRegression()
best_model(lr)

rf=RandomForestClassifier()
best_model(rf)

dt=DecisionTreeClassifier()
best_model(dt)

nb=GaussianNB()
best_model(nb)

knn=KNeighborsClassifier()
best_model(knn)


svc=SVC()
best_model(svc)

In [ ]:
Final=pd.DataFrame()
Final['Model']=['LogisticRegression','DecisionTreeClassifier','SVC','KNeighborsClassifier','GaussianNB','RandomForestClassifier']

Final["Accuracy Score"]=score
Final['Cross Val']=CV_score
Final['Standard Deviation']=SD
Final['Diff']=Final["Accuracy Score"]-Final['Cross Val']
Final['F_1 Score']=f_1
Final

# Hyperparameter Tuning

In [ ]:
lg_1=LogisticRegression()
param={'penalty':['l2','l1','elasticnet'],'C':[.001,.01,.1,1,10],'solver':['newton-cg','lbfgs','liblinear','sag','saga'],'max_iter':[50,100,200]}

grid=GridSearchCV(estimator=lg_1,param_grid=param,scoring='accuracy',n_jobs=-1)

grid.fit(X_train_norm,y_train)

grid.best_params_

In [ ]:
lgm=LogisticRegression(C=10,max_iter=200,penalty='l1',solver='saga')
lgm.fit(X_train_norm,y_train)
pred=lgm.predict(X_test_norm)
print('Final Accuracy_score :',accuracy_score(pred,y_test))
print('\n')
print('Final f_1 score :',f1_score(pred,y_test))
print('\n')
print('Final roc_auc score :',roc_auc_score(pred,y_test))
print('\n')
print('Final classification Report :',classification_report(pred,y_test))
print('\n')
print('Final confusion Matrix :',confusion_matrix(pred,y_test))

# Saving Model

In [ ]:
import pickle
filename='LeadScoring'
pickle.dump(lgm,open(filename,'wb'))

# Loading Model

In [ ]:
loaded_model=pickle.load(open(filename,'rb'))

In [ ]:
loaded_model.score(X_test_norm,y_test)

loaded_model.predict_proba([[]])

loaded_model.predict([values1,values2,values3,.....])